# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kuhmo,64.1333,29.5167,-9.92,96,98,2.60,FI,1702503619
1,1,tura,25.5198,90.2201,12.84,64,0,1.06,IN,1702503619
2,2,waitangi,-43.9535,-176.5597,12.12,79,65,5.36,NZ,1702503619
3,3,invercargill,-46.4000,168.3500,12.77,62,86,5.66,NZ,1702503585
4,4,chos malal,-37.3781,-70.2709,27.30,9,3,6.53,AR,1702503620


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True, 
                                      tiles = "OSM", 
                                      frame_width = 750, 
                                      frame_height = 550,
                                      size = "Humidity",
                                      color = "City",
                                      alpha = 0.5)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
# It says to find MY ideal weather conditions. For reference I am a redhead with freckles. I cant be getting too sunburnt out here.

ideal_weather_df = city_data_df.loc[(city_data_df["Cloudiness"] >= 50) & 
                                    (city_data_df["Max Temp"] <= 26) & 
                                    (city_data_df["Max Temp"] >= 18) 
                                    ]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,blackmans bay,-43.0167,147.3167,18.07,84,68,1.41,AU,1702503620
11,11,adamstown,-25.0660,-130.1015,23.45,83,99,4.75,PN,1702503621
15,15,new norfolk,-42.7826,147.0587,18.72,90,66,2.32,AU,1702503622
34,34,vega de alatorre,20.0333,-96.6333,21.00,81,100,4.19,MX,1702503627
40,40,karoi,-16.8099,29.6925,18.98,90,100,2.17,ZW,1702503628
...,...,...,...,...,...,...,...,...,...,...
533,533,ankazobe,-18.3167,47.1167,20.17,85,100,2.99,MG,1702503681
536,536,altagracia de orituco,9.8601,-66.3814,24.85,76,58,2.89,VE,1702503682
544,544,lusambo,-4.9667,23.4500,20.13,99,100,0.74,CD,1702503684
553,553,palopo,-2.9996,120.1921,25.20,91,100,0.92,ID,1702503687


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng" , "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

/var/folders/qp/cryp6xxx017by8b4yxwh79tc0000gp/T/ipykernel_59591/951982974.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
7,blackmans bay,AU,-43.0167,147.3167,84,
11,adamstown,PN,-25.0660,-130.1015,83,
15,new norfolk,AU,-42.7826,147.0587,90,
34,vega de alatorre,MX,20.0333,-96.6333,81,
40,karoi,ZW,-16.8099,29.6925,90,
...,...,...,...,...,...,...
533,ankazobe,MG,-18.3167,47.1167,85,
536,altagracia de orituco,VE,9.8601,-66.3814,76,
544,lusambo,CD,-4.9667,23.4500,99,
553,palopo,ID,-2.9996,120.1921,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accomodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
vega de alatorre - nearest hotel: No hotel found
karoi - nearest hotel: No hotel found
kisangani - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
saint-leu - nearest hotel: No hotel found
manakara - nearest hotel: No hotel found
inongo - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
port-vila - nearest hotel: No hotel found
yen vinh - nearest hotel: No hotel found
islamkot - nearest hotel: No hotel found
nishinoomote - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
cannonvale - nearest hotel: No hotel found
cidreira - nearest hotel: No hotel found
danao - nearest hotel: No hotel found
mitu - nearest hotel: No hotel found
saurimo - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
port saint john's - nearest hotel: No hotel fo

,City,Country,Lat,Lng,Humidity,Hotel Name
7,blackmans bay,AU,-43.0167,147.3167,84,No hotel found
11,adamstown,PN,-25.0660,-130.1015,83,No hotel found
15,new norfolk,AU,-42.7826,147.0587,90,No hotel found
34,vega de alatorre,MX,20.0333,-96.6333,81,No hotel found
40,karoi,ZW,-16.8099,29.6925,90,No hotel found
...,...,...,...,...,...,...
533,ankazobe,MG,-18.3167,47.1167,85,No hotel found
536,altagracia de orituco,VE,9.8601,-66.3814,76,No hotel found
544,lusambo,CD,-4.9667,23.4500,99,No hotel found
553,palopo,ID,-2.9996,120.1921,91,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points("Lng",
                                    "Lat",
                                    geo = True,
                                    tiles = "OSM",
                                    frame_width = 750,
                                    frame_height = 550,
                                    color = "City",
                                    alpha = 0.5,
                                    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)